In [1]:
# load libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from scipy import stats

import warnings

# suppress all warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

In [2]:
# specify file paths
train_df1 = "train_subset.csv"
train_targets1 = "train_targets.csv"
train_df2 = "train_subset_375.csv"
train_targets2 = "targets_375.csv"
test_df = "test.csv"

# read in files
X_train1 = pd.read_csv(train_df1)
y_train1 = pd.read_csv(train_targets1)['AAC']
X_train2 = pd.read_csv(train_df2)
y_train2 = pd.read_csv(train_targets2)['AAC']
X_test1 = pd.read_csv(test_df)
X_test2 = pd.read_csv(test_df)# specify file paths
train_df1 = "train_subset.csv"
train_targets1 = "train_targets.csv"
train_df2 = "train_subset_375.csv"
train_targets2 = "targets_375.csv"
test_df = "test.csv"

# read in files
X_train1 = pd.read_csv(train_df1)
y_train1 = pd.read_csv(train_targets1)['AAC']
X_train2 = pd.read_csv(train_df2)
y_train2 = pd.read_csv(train_targets2)['AAC']
X_test1 = pd.read_csv(test_df)
X_test2 = pd.read_csv(test_df)

# filter to keep only relevant genes
X_test1 = X_test1[X_train1.columns]
X_test2 = X_test2[X_train2.columns]

print(X_train1.shape)
print(y_train1.shape)
print(X_train2.shape)
print(y_train2.shape)
print(X_test1.shape)
print(X_test2.shape)

# filter to keep only relevant genes
X_test1 = X_test1[X_train1.columns]
X_test2 = X_test2[X_train2.columns]

print(X_train1.shape)
print(y_train1.shape)
print(X_train2.shape)
print(y_train2.shape)
print(X_test1.shape)
print(X_test2.shape)

(742, 457)
(742,)
(662, 510)
(662,)
(304, 457)
(304, 510)
(742, 457)
(742,)
(662, 510)
(662,)
(304, 457)
(304, 510)


In [3]:
# Initialize models with slightly adjusted hyperparameters
enALL = linear_model.ElasticNet(alpha=1.2, l1_ratio=1, max_iter=1000)
en375 = linear_model.ElasticNet(alpha=8, l1_ratio=0.25, max_iter=1000)

# Fit models
enALL.fit(X_train1, y_train1)
en375.fit(X_train2, y_train2)

# Get predictions
pred1 = enALL.predict(X_test1)
pred2 = en375.predict(X_test2)

# Weighted average predictions (tune weights if needed)
y_pred = (pred1 * 0.25 + pred2 * 0.75) / (0.25+0.75)

# Save predictions
sample_ids = [f"TS{i}" for i in range(1, len(y_pred) + 1)]
predictions = pd.DataFrame({
    'sampleId': sample_ids,
    'AAC': y_pred
})
predictions.to_csv("optimized_en-ensemble-pred.csv", index=False)